# SEO-Helperskript
### Liesst die Bilder im INPUT Folder, generiert SEO-optimierte ALT und TITLE Tags und verschiebt die bearbeiteten Bilder.
* Nutzt Gemini 1.5 Pro
* time.sleep(32) relativ hoch, da API free tier. (könnte 1.5s bei pad plan)

In [22]:
!bash streamlit run seo_app.py

import-im6.q16: unable to open X server `' @ error/import.c/ImportImageCommand/359.
import-im6.q16: unable to open X server `' @ error/import.c/ImportImageCommand/359.
from: can't read /var/mail/streamlit.cli
/opt/conda/envs/anaconda-ai-2024.04-py310/bin/streamlit: streamlit: line 10: syntax error near unexpected token `('
/opt/conda/envs/anaconda-ai-2024.04-py310/bin/streamlit: streamlit: line 10: `    sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])'


In [21]:
!pip install streamlit google-generativeai Pillow python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  You can safely remove it manually.
  Attempting uninstall: grpcio-status
    Found e

In [4]:
# import os # Stelle sicher, dass dein Google API Key als Umgebungsvariable gesetzt ist
# os.environ["GOOGLE_API_KEY"] = "XYZ"

In [16]:
# -*- coding: utf-8 -*-

import os
import google.generativeai as genai
from PIL import Image
from pathlib import Path
from dotenv import load_dotenv
import time # Import für eine kleine Pause

# 1. Lade Umgebungsvariablen (API Key) und konfiguriere Gemini
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Fehler: GOOGLE_API_KEY nicht in der .env Datei gefunden.")
    # Beenden oder alternative Eingabe hier handhaben
    exit() # Beendet das Skript, wenn kein Key vorhanden ist
else:
    genai.configure(api_key=api_key)

# 2. Definiere die Funktion zur Tag-Generierung (unverändert)
def generate_image_tags(image_path: str, model_name: str = "gemini-1.5-pro-latest") -> tuple[str | None, str | None]:
    """
    Nimmt einen Bildpfad, sendet das Bild an Gemini und gibt
    SEO-optimierte title- und alt-Tags zurück.

    Args:
        image_path: Pfad zur Bilddatei.
        model_name: Name des zu verwendenden Gemini Modells.

    Returns:
        Ein Tupel (title_tag, alt_tag) oder (None, None) bei Fehlern.
    """
    print(f"  Verarbeite: {Path(image_path).name}") # Nur Dateiname für Übersicht
    image_path_obj = Path(image_path) # Sicherstellen, dass es ein Path-Objekt ist
    if not image_path_obj.is_file():
        print(f"  Fehler: Bilddatei nicht gefunden unter {image_path_obj}")
        return None, None

    try:
        # Lade das Bild
        img = Image.open(image_path_obj)

        # Wähle das multimodale Modell
        model = genai.GenerativeModel(model_name)

        # Definiere den Prompt für Gemini
        prompt = """
        Analysiere das folgende Bild sorgfältig.
        Deine Aufgabe ist es, SEO-optimierte HTML-Attribute für dieses Bild zu generieren:
        1. Ein 'alt'-Attribut (Alternativtext)
        2. Ein 'title'-Attribut

        Beachte dabei die aktuellen SEO Best Practices:
        - Das 'alt'-Attribut muss das Bild präzise und prägnant beschreiben. Es ist entscheidend für Barrierefreiheit (Screenreader) und das Verständnis des Bildinhalts durch Suchmaschinen. Beschreibe Objekte, Personen, Aktionen und ggf. Text im Bild. Vermeide Keyword-Stuffing.
        - Das 'title'-Attribut wird oft als Tooltip beim Überfahren mit der Maus angezeigt. Es kann zusätzliche kontextbezogene Informationen liefern, die über die reine Beschreibung des 'alt'-Attributs hinausgehen, sollte aber ebenfalls relevant sein.

        Gib *nur* die beiden Attribute im folgenden Format zurück, ohne zusätzliche Erklärungen oder Formatierungen:

        ALT: [Hier der generierte Alt-Text]
        TITLE: [Hier der generierte Title-Text]
        """

        # Sende die Anfrage an die Gemini API (Bild + Text Prompt)
        # print("  Sende Anfrage an Gemini...") # Kann man für weniger Output auskommentieren
        response = model.generate_content([prompt, img], request_options={"timeout": 120}) # Timeout erhöht

        # Kurze Pause, um Rate Limits der API vorzubeugen (optional, aber empfohlen)
        time.sleep(32) # Warte x Sekunden vor der nächsten Anfrage (relativ hoch, da free tier)

        # Verarbeite die Antwort
        # print("  Antwort von Gemini erhalten.") # Kann man auskommentieren
        generated_text = response.text.strip()

        # Extrahiere alt und title Tags aus der Antwort
        alt_tag = None
        title_tag = None
        lines = generated_text.split('\n')
        for line in lines:
            if line.upper().startswith("ALT:"):
                alt_tag = line[len("ALT:"):].strip()
            elif line.upper().startswith("TITLE:"):
                title_tag = line[len("TITLE:"):].strip()

        if alt_tag and title_tag:
            # print("  Tags erfolgreich extrahiert.") # Kann man auskommentieren
            return title_tag, alt_tag
        else:
            print(f"  Fehler: Konnte Tags nicht aus Gemini-Antwort extrahieren für {Path(image_path).name}.")
            print(f"  Rohe Antwort: {generated_text}")
            return None, None

    except FileNotFoundError:
        print(f"  Fehler: Bilddatei (während Verarbeitung) nicht gefunden unter {image_path_obj}")
        return None, None
    except Exception as e:
        print(f"  Fehler während der Gemini-Verarbeitung für {Path(image_path).name}: {e}")
        try:
             if response: # Nur wenn response existiert
                 print(f"  Prompt Feedback: {response.prompt_feedback}")
        except Exception:
             pass
        return None, None

# --- NEU: Batch-Verarbeitung ---

# 3. Definiere die Verzeichnisse (relativ zum CWD)
#    Da dein CWD /.../SEO_Helper ist, sind das die direkten Unterordnernamen
input_folder_name = "pics_to_process"
output_folder_name = "pics_done"

# Erstelle Path-Objekte für die Verzeichnisse
base_dir = Path(os.getcwd()) # Nimmt das aktuelle Arbeitsverzeichnis als Basis
input_dir = base_dir / input_folder_name
output_dir = base_dir / output_folder_name

print(f"Basisverzeichnis: {base_dir}")
print(f"Input-Verzeichnis: {input_dir}")
print(f"Output-Verzeichnis: {output_dir}")

# 4. Stelle sicher, dass das Output-Verzeichnis existiert
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output-Verzeichnis '{output_dir.name}' sichergestellt.")

# 5. Definiere gültige Bild-Endungen (Kleinbuchstaben!)
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']

# 6. Sammle alle Bilddateien im Input-Verzeichnis
image_files_to_process = []
print(f"\nSuche nach Bildern in '{input_dir.name}'...")
for item in input_dir.iterdir():
    # Prüfe, ob es eine Datei ist und die Endung passt (Groß-/Kleinschreibung ignorieren)
    if item.is_file() and item.suffix.lower() in image_extensions:
        image_files_to_process.append(item)

if not image_files_to_process:
    print("Keine Bilddateien im Input-Verzeichnis gefunden.")
else:
    print(f"{len(image_files_to_process)} Bilddatei(en) gefunden. Starte Verarbeitung...")
    processed_count = 0
    failed_count = 0
    results = {} # Optional: Dictionary zum Speichern der Ergebnisse

    # 7. Verarbeite jede gefundene Bilddatei
    for image_file_path in image_files_to_process:
        print("-" * 20) # Trennlinie für Übersicht
        try:
            # Generiere die Tags
            # Wichtig: Übergebe den Pfad als String, wie von der Funktion erwartet
            title, alt = generate_image_tags(str(image_file_path))

            # Prüfe, ob die Generierung erfolgreich war
            if title and alt:
                print(f"  Erfolg: Tags für '{image_file_path.name}' generiert.")
                print(f"    ALT:   {alt}")
                print(f"    TITLE: {title}")

                # Optional: Speichere Ergebnisse
                results[image_file_path.name] = {'title': title, 'alt': alt}

                # Verschiebe die Datei in den Output-Ordner
                destination_path = output_dir / image_file_path.name
                try:
                    image_file_path.rename(destination_path)
                    print(f"  Datei '{image_file_path.name}' verschoben nach '{output_dir.name}'.")
                    processed_count += 1
                except OSError as move_error:
                    print(f"  FEHLER beim Verschieben von '{image_file_path.name}': {move_error}")
                    # Optional: Entscheiden, ob man den Fehler zählt oder anders behandelt
                    failed_count += 1 # Zählen als Fehler, da nicht verschoben

            else:
                print(f"  Fehler: Konnte keine Tags für '{image_file_path.name}' generieren. Datei wird nicht verschoben.")
                failed_count += 1

        except Exception as e:
            print(f"  Unerwarteter FEHLER bei der Verarbeitung von '{image_file_path.name}': {e}")
            failed_count += 1

    # 8. Gib eine Zusammenfassung aus
    print("\n" + "=" * 30)
    print("Batch-Verarbeitung abgeschlossen.")
    print(f"Erfolgreich verarbeitet und verschoben: {processed_count}")
    print(f"Fehlgeschlagen / Nicht verschoben:      {failed_count}")
    print("=" * 30)

    # Optional: Zeige gesammelte Ergebnisse
    # print("\nGesammelte Ergebnisse:")
    # for filename, tags in results.items():
    #    print(f"- {filename}: ALT='{tags['alt']}', TITLE='{tags['title']}'")

Basisverzeichnis: /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper
Input-Verzeichnis: /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper/pics_to_process
Output-Verzeichnis: /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper/pics_done
Output-Verzeichnis 'pics_done' sichergestellt.

Suche nach Bildern in 'pics_to_process'...
4 Bilddatei(en) gefunden. Starte Verarbeitung...
--------------------
  Verarbeite: 366527889898.jpeg
  Erfolg: Tags für '366527889898.jpeg' generiert.
    ALT:   Bengal-Katze spielt auf dem Rücken liegend auf einer Decke
    TITLE: Verspielte Bengal-Katze entspannt sich auf dem Rücken
  Datei '366527889898.jpeg' verschoben nach 'pics_done'.
--------------------
  Verarbeite: 9783644020160.webp
  Erfolg: Tags für '9783644020160.webp' generiert.
    ALT:   Buchcover von "Zypressen Sommer" von Teresa Simon.  Es zeigt einen Olivenzweig, eine toskanische Landschaft mit Häusern und eine Frau in einem gepunkteten Kleid.
    TITLE: Zypressen Sommer - Roman von Sp

In [15]:
# -*- coding: utf-8 -*-

import google.generativeai as genai
from PIL import Image
from pathlib import Path
from dotenv import load_dotenv

# 1. Lade Umgebungsvariablen (API Key)
#    Erstelle eine .env Datei im selben Verzeichnis wie dein Notebook
#    und füge deinen API Key hinzu: GOOGLE_API_KEY="DEIN_API_KEY"
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Fehler: GOOGLE_API_KEY nicht in der .env Datei gefunden.")
    # Alternativ: Direkte Eingabe (weniger sicher)
    # api_key = input("Bitte gib deinen Google API Key ein: ")
else:
    genai.configure(api_key=api_key)

# 2. Definiere die Funktion zur Tag-Generierung
def generate_image_tags(image_path: str, model_name: str = "gemini-1.5-pro-latest") -> tuple[str | None, str | None]:
    """
    Nimmt einen Bildpfad, sendet das Bild an Gemini und gibt
    SEO-optimierte title- und alt-Tags zurück.

    Args:
        image_path: Pfad zur Bilddatei.
        model_name: Name des zu verwendenden Gemini Modells.

    Returns:
        Ein Tupel (title_tag, alt_tag) oder (None, None) bei Fehlern.
    """
    print(f"Verarbeite Bild: {image_path}")
    image_path = Path(image_path)
    if not image_path.is_file():
        print(f"Fehler: Bilddatei nicht gefunden unter {image_path}")
        return None, None

    try:
        # Lade das Bild
        img = Image.open(image_path)

        # Wähle das multimodale Modell
        # 'gemini-pro-vision' ist auch eine Option, aber 1.5 ist oft leistungsfähiger
        model = genai.GenerativeModel(model_name)

        # Definiere den Prompt für Gemini
        # Wichtig: Gib klare Anweisungen und erwähne die SEO Best Practices
        prompt = """
        Analysiere das folgende Bild sorgfältig.
        Deine Aufgabe ist es, SEO-optimierte HTML-Attribute für dieses Bild zu generieren:
        1. Ein 'alt'-Attribut (Alternativtext)
        2. Ein 'title'-Attribut

        Beachte dabei die aktuellen SEO Best Practices:
        - Das 'alt'-Attribut muss das Bild präzise und prägnant beschreiben. Es ist entscheidend für Barrierefreiheit (Screenreader) und das Verständnis des Bildinhalts durch Suchmaschinen. Beschreibe Objekte, Personen, Aktionen und ggf. Text im Bild. Vermeide Keyword-Stuffing.
        - Das 'title'-Attribut wird oft als Tooltip beim Überfahren mit der Maus angezeigt. Es kann zusätzliche kontextbezogene Informationen liefern, die über die reine Beschreibung des 'alt'-Attributs hinausgehen, sollte aber ebenfalls relevant sein.

        Gib *nur* die beiden Attribute im folgenden Format zurück, ohne zusätzliche Erklärungen oder Formatierungen:

        ALT: [Hier der generierte Alt-Text]
        TITLE: [Hier der generierte Title-Text]
        """

        # Sende die Anfrage an die Gemini API (Bild + Text Prompt)
        print("Sende Anfrage an Gemini...")
        response = model.generate_content([prompt, img])

        # Verarbeite die Antwort
        print("Antwort von Gemini erhalten.")
        generated_text = response.text.strip()

        # Extrahiere alt und title Tags aus der Antwort
        alt_tag = None
        title_tag = None
        lines = generated_text.split('\n')
        for line in lines:
            if line.upper().startswith("ALT:"):
                alt_tag = line[len("ALT:"):].strip()
            elif line.upper().startswith("TITLE:"):
                title_tag = line[len("TITLE:"):].strip()

        if alt_tag and title_tag:
            print("Tags erfolgreich extrahiert.")
            return title_tag, alt_tag
        else:
            print("Fehler: Konnte Tags nicht aus der Gemini-Antwort extrahieren.")
            print("Rohe Antwort:", generated_text)
            return None, None

    except FileNotFoundError:
        print(f"Fehler: Bilddatei nicht gefunden unter {image_path}")
        return None, None
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")
        # Oftmals hilft es, die response.prompt_feedback zu prüfen bei API Fehlern
        try:
             print(f"Prompt Feedback: {response.prompt_feedback}")
        except Exception:
             pass # Falls response noch nicht existiert oder Fehler hat
        return None, None



# --- DEBUGGING START ---
image_file = 'pics_to_process/9783644019270.webp'
print(f"Aktuelles Arbeitsverzeichnis (CWD): {os.getcwd()}")

# Erstelle das Path-Objekt
img_path_obj = Path(image_file)

print(f"Versuche Pfad: {image_file}")
print(f"Absoluter Pfad versucht: {img_path_obj.resolve()}") # Zeigt den vollen Pfad, den Python verwendet
print(f"Existiert die Datei laut Python? {img_path_obj.exists()}")
print(f"Ist es eine Datei? {img_path_obj.is_file()}")
# --- DEBUGGING END ---


# 3. Beispielaufruf
#    Dein angepasster Code - leicht vereinfacht für die Prüfung

# Überprüfe NUR, ob die Datei existiert
if not img_path_obj.is_file(): # Sicherere Prüfung als nur exists()
     print("\nFEHLER: Datei nicht gefunden oder kein reguläres File.")
     print(f"Stelle sicher, dass die Datei hier existiert: {img_path_obj.resolve()}")
     print(f"Und dass das Skript vom richtigen Verzeichnis läuft (siehe CWD oben).")
     title, alt = None, None # Setze auf None
else:
    print("\nDatei gefunden. Versuche Tags zu generieren...")
    # Rufe die Funktion auf (stelle sicher, dass sie vorher definiert wurde)
    # Füge einen try-except Block hinzu, falls der Fehler IN der Funktion liegt
    try:
        title, alt = generate_image_tags(str(img_path_obj)) # Übergib den Pfad als String
    except Exception as e:
        print(f"\nFEHLER während generate_image_tags: {e}")
        title, alt = None, None


# 4. Gib die Ergebnisse aus
if title and alt:
    print("\n--- Generierte SEO Tags ---")
    print(f"HTML Title Attribut: \"{title}\"")
    print(f"HTML Alt Attribut:   \"{alt}\"")
    print("\n--- Beispiel HTML Code ---")
    # Verwende den ursprünglichen relativen Pfad für das Beispiel-HTML src
    print(f'<img src="{image_file}" alt="{alt}" title="{title}">')
else:
    print("\nKonnte keine Tags für das Bild generieren (siehe Fehlermeldungen oben).")







Aktuelles Arbeitsverzeichnis (CWD): /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper
Versuche Pfad: pics_to_process/9783644019270.webp
Absoluter Pfad versucht: /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper/pics_to_process/9783644019270.webp
Existiert die Datei laut Python? True
Ist es eine Datei? True

Datei gefunden. Versuche Tags zu generieren...
Verarbeite Bild: pics_to_process/9783644019270.webp
Sende Anfrage an Gemini...
Antwort von Gemini erhalten.
Tags erfolgreich extrahiert.

--- Generierte SEO Tags ---
HTML Title Attribut: "Buchcover: Das Pubquiz für einsame Herzen von Lauren Farnsworth - Roman"
HTML Alt Attribut:   "Buchcover von "Das Pubquiz für einsame Herzen" von Lauren Farnsworth. Darstellung von Personen an einer Bar mit Quizbögen."

--- Beispiel HTML Code ---
<img src="pics_to_process/9783644019270.webp" alt="Buchcover von "Das Pubquiz für einsame Herzen" von Lauren Farnsworth. Darstellung von Personen an einer Bar mit Quizbögen." title="Buchcover: Das Pubq

## HILFSSKRIPT: Bilder von 'pics_done' nach 'pics_to_process' zurückverschieben ---

In [19]:
# --- HILFSSKRIPT: Bilder von 'pics_done' nach 'pics_to_process' zurückverschieben ---

import os
from pathlib import Path
import shutil # Sicherere Alternative zum Verschieben, falls rename Probleme macht

print("Starte Skript zum Zurücksetzen der Testbilder...")

# 1. Definiere die Verzeichnisse (relativ zum CWD)
#    Sollten dieselben Namen sein wie im Hauptskript
source_folder_name = "pics_done"    # Der Ordner, AUS dem verschoben wird
dest_folder_name = "pics_to_process" # Der Ordner, IN den verschoben wird

# Erstelle Path-Objekte für die Verzeichnisse
base_dir = Path(os.getcwd()) # Nimmt das aktuelle Arbeitsverzeichnis als Basis
source_dir = base_dir / source_folder_name
dest_dir = base_dir / dest_folder_name

print(f"Aktuelles Arbeitsverzeichnis: {base_dir}")
print(f"Quell-Verzeichnis (pics_done): {source_dir}")
print(f"Ziel-Verzeichnis (pics_to_process): {dest_dir}")

# 2. Prüfe, ob das Quell-Verzeichnis existiert
if not source_dir.is_dir():
    print(f"\nFEHLER: Quell-Verzeichnis '{source_dir.name}' nicht gefunden.")
    print("Keine Dateien zum Verschieben vorhanden.")
else:
    # 3. Stelle sicher, dass das Ziel-Verzeichnis existiert (falls es gelöscht wurde)
    dest_dir.mkdir(parents=True, exist_ok=True)
    print(f"Ziel-Verzeichnis '{dest_dir.name}' sichergestellt.")

    # 4. Finde alle Dateien im Quell-Verzeichnis
    files_to_move = [item for item in source_dir.iterdir() if item.is_file()]

    if not files_to_move:
        print(f"\nKeine Dateien im Verzeichnis '{source_dir.name}' gefunden.")
    else:
        print(f"\nVerschiebe {len(files_to_move)} Datei(en) von '{source_dir.name}' nach '{dest_dir.name}'...")
        moved_count = 0
        error_count = 0

        # 5. Verschiebe jede Datei
        for file_path in files_to_move:
            source_path = file_path
            destination_path = dest_dir / file_path.name # Zielpfad mit Dateinamen

            try:
                # Versuche die Datei zu verschieben
                # Alternative 1: Path.rename (einfacher, kann bei Laufwerksgrenzen fehlschlagen)
                source_path.rename(destination_path)

                # Alternative 2: shutil.move (robuster, handhabt Laufwerksgrenzen)
                # shutil.move(str(source_path), str(destination_path)) # Pfade als Strings übergeben

                print(f"  - '{file_path.name}' verschoben.")
                moved_count += 1
            except Exception as e:
                print(f"  - FEHLER beim Verschieben von '{file_path.name}': {e}")
                error_count += 1

        # 6. Gib eine Zusammenfassung aus
        print("\n" + "=" * 30)
        print("Verschieben abgeschlossen.")
        print(f"Erfolgreich verschoben: {moved_count}")
        print(f"Fehler beim Verschieben: {error_count}")
        print("=" * 30)

Starte Skript zum Zurücksetzen der Testbilder...
Aktuelles Arbeitsverzeichnis: /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper
Quell-Verzeichnis (pics_done): /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper/pics_done
Ziel-Verzeichnis (pics_to_process): /home/cd0ebd20-6f7a-47e0-8548-4e2d7bb904f7/SEO_Helper/pics_to_process
Ziel-Verzeichnis 'pics_to_process' sichergestellt.

Verschiebe 3 Datei(en) von 'pics_done' nach 'pics_to_process'...
  - '366527889898.jpeg' verschoben.
  - '9783644020160.webp' verschoben.
  - '9783644019270.webp' verschoben.

Verschieben abgeschlossen.
Erfolgreich verschoben: 3
Fehler beim Verschieben: 0
